# 🤖 **Módulo 5: Agents - Os Funcionários Inteligentes**

## **Aula 5.1: Agents Básicos - Delegando Tarefas**

---

### **Tá, mas o que são Agents?**

Imagina que você é um **chefe de uma empresa** e tem funcionários especializados:

**Sem Agents**: Você tem que fazer tudo sozinho - pesquisar, calcular, escrever, analisar...
**Com Agents**: Você delega para funcionários especializados que sabem usar ferramentas!

**Agent** = IA que pode **tomar decisões** e **usar ferramentas** para completar tarefas.

### **Por que Agents são Poderosos?**

**Sem Agents**: IA só conversa, não pode fazer ações
**Com Agents**: IA pode pesquisar, calcular, executar código, etc.

É como a diferença entre **conversar com alguém** e **ter um assistente que pode usar computador, calculadora, internet...**

---

**🖼️ Sugestão de imagem**: Um diagrama mostrando um chefe delegando tarefas para funcionários especializados

### **Setup Inicial - Preparando o Terreno**

In [ ]:
# Importando bibliotecas para Agents
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.memory import ConversationBufferMemory

# Carregando variáveis
load_dotenv()

# Modelo
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7,
    api_key=os.getenv('OPENAI_API_KEY')  # Configure sua API key no Colab
)

print("🤖 Setup completo para Agents!")
print(f"🚀 Modelo: {llm.model_name}")

### **Criando Nossa Primeira Ferramenta**

Vamos criar um **agente de pesquisa** que pode buscar informações na internet:

In [ ]:
# Criando ferramenta de pesquisa
# Como dar uma calculadora para o funcionário

from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import Tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Ferramenta de pesquisa
search_tool = DuckDuckGoSearchRun()

# NOVO: Criar ferramentas com descrição
tools = [
    Tool(
        name="Pesquisa na Internet",
        func=search_tool.run,
        description="Use para buscar informações atualizadas na internet"
    )
]

# Memory para o agente
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# NOVO: Template para o agente
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente útil que pode pesquisar na internet."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# NOVO: Criar agente com sintaxe moderna
agent = create_openai_functions_agent(llm, tools, prompt)

# NOVO: Executor do agente
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

print("�� Agente de pesquisa criado!")
print("🌐 Pode buscar informações na internet")

In [ ]:
# Testando o agente de pesquisa
# Vamos ver ele pesquisando!

print("�� TESTE: AGENTE DE PESQUISA")
print("=" * 50)

perguntas = [
    "Qual é a capital do Brasil?",
    "Quem é o presidente atual do Brasil?",
    "Qual a população de São Paulo?"
]

for i, pergunta in enumerate(perguntas, 1):
    print(f"\n❓ Pergunta {i}: {pergunta}")

    try:
        # NOVO: Usar agent_executor.invoke() em vez de agent.run()
        resposta = agent_executor.invoke({"input": pergunta})

        # NOVO: Acessar o conteúdo da resposta
        print(f"🤖 Resposta: {resposta['output'][:200]}...")

    except Exception as e:
        print(f"❌ Erro: {e}")

    print("-" * 40)

## **Aula 5.2: Ferramentas Customizadas - Swiss Army Knife da IA**

### **Criando Ferramentas Personalizadas**

Vamos criar ferramentas customizadas. É como dar **ferramentas específicas** para cada funcionário:

In [ ]:
# Criando ferramentas customizadas
# Como dar ferramentas específicas para cada tarefa

from langchain.tools import BaseTool
from typing import Optional
import math

# Ferramenta de cálculo
class CalculadoraTool(BaseTool):
    name = "calculadora"
    description = "Use para fazer cálculos matemáticos"

    def _run(self, query: str) -> str:
        try:
            # Avalia expressão matemática com segurança
            allowed_names = {
                'abs': abs, 'round': round, 'min': min, 'max': max,
                'sum': sum, 'pow': pow, 'sqrt': math.sqrt
            }
            result = eval(query, {"__builtins__": {}}, allowed_names)
            return f"Resultado: {result}"
        except Exception as e:
            return f"Erro no cálculo: {e}"

# Ferramenta de análise de texto
class AnalisadorTextoTool(BaseTool):
    name = "analisador_texto"
    description = "Use para analisar textos (contar palavras, caracteres, etc.)"

    def _run(self, texto: str) -> str:
        palavras = len(texto.split())
        caracteres = len(texto)
        linhas = len(texto.split('\n'))

        return f"Análise do texto:\n- Palavras: {palavras}\n- Caracteres: {caracteres}\n- Linhas: {linhas}"

print("🔧 Ferramentas customizadas criadas!")
print("🧮 Calculadora e Analisador de Texto prontos")

In [ ]:
# Criando agente com ferramentas customizadas
# Como montar uma equipe completa

tools_avancadas = [
    Tool(
        name="Pesquisa na Internet",
        func=search_tool.run,
        description="Use para buscar informações atualizadas"
    ),
    Tool(
        name="Calculadora",
        func=CalculadoraTool()._run,
        description="Use para fazer cálculos matemáticos"
    ),
    Tool(
        name="Analisador de Texto",
        func=AnalisadorTextoTool()._run,
        description="Use para analisar textos"
    )
]

# NOVO: Criar agente com sintaxe moderna
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Template para o agente avançado
prompt_avancado = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente inteligente que pode pesquisar, calcular e analisar textos. Use as ferramentas disponíveis quando necessário."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# NOVO: Criar agente com sintaxe moderna
agent_avancado = create_openai_functions_agent(llm, tools_avancadas, prompt_avancado)

# NOVO: Executor do agente
agent_executor_avancado = AgentExecutor(
    agent=agent_avancado,
    tools=tools_avancadas,
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=True
)

print("🤖 Agente avançado criado!")
print("🛠️  Pode pesquisar, calcular e analisar textos")

In [ ]:
# Testando o agente avançado
# Vamos ver ele usando todas as ferramentas!

print("�� TESTE: AGENTE AVANÇADO")
print("=" * 50)

tarefas = [
    "Pesquise sobre a população do Brasil e calcule quantas pessoas isso representa por km²",
    "Analise este texto: 'Python é uma linguagem de programação incrível para iniciantes.'",
    "Calcule a raiz quadrada de 144 e depois pesquise sobre o número 12 na matemática"
]

for i, tarefa in enumerate(tarefas, 1):
    print(f"\n📋 Tarefa {i}: {tarefa}")

    try:
        # NOVO: Usar agent_executor_avancado.invoke() em vez de agent_avancado.run()
        resultado = agent_executor_avancado.invoke({"input": tarefa})

        # NOVO: Acessar o conteúdo da resposta
        print(f"�� Resultado: {resultado['output'][:300]}...")

    except Exception as e:
        print(f"❌ Erro: {e}")

    print("-" * 40)

## **Aula 5.3: Agente Financeiro - Tomada de Decisão Inteligente**

### **Criando um Agente Especializado**

Vamos criar um **agente financeiro** que pode analisar investimentos e dar conselhos:

In [ ]:
# Ferramenta de análise financeira
class AnaliseFinanceiraTool(BaseTool):
    name = "analise_financeira"
    description = "Use para analisar investimentos e dar conselhos financeiros"

    def _run(self, pergunta: str) -> str:
        # Simulação de análise financeira
        if "cdb" in pergunta.lower():
            return "CDB é um investimento de renda fixa seguro. Rentabilidade: 100-110% do CDI. Ideal para conservadores."
        elif "ações" in pergunta.lower():
            return "Ações são investimentos de renda variável. Maior risco, maior potencial de retorno. Ideal para arrojados."
        elif "tesouro" in pergunta.lower():
            return "Tesouro Direto é o investimento mais seguro do Brasil. Garantido pelo governo federal."
        else:
            return "Para investimentos, considere seu perfil de risco e objetivos. Diversificação é fundamental."

# Ferramenta de cálculo de juros compostos
class JurosCompostosTool(BaseTool):
    name = "juros_compostos"
    description = "Use para calcular juros compostos e projeções financeiras"

    def _run(self, query: str) -> str:
        try:
            # Exemplo: calcular juros compostos
            if "calcular" in query.lower():
                # Simulação simples
                principal = 1000
                taxa = 0.01  # 1% ao mês
                tempo = 12   # 12 meses

                montante = principal * (1 + taxa) ** tempo
                return f"Investimento de R$ {principal} por {tempo} meses a {taxa*100}% ao mês = R$ {montante:.2f}"
            else:
                return "Use 'calcular' para fazer projeções financeiras"
        except:
            return "Erro no cálculo financeiro"

print("💰 Ferramentas financeiras criadas!")
print("📊 Análise financeira e cálculos de juros compostos")

In [ ]:
# Criando agente financeiro
# Como contratar um consultor financeiro inteligente

tools_financeiras = [
    Tool(
        name="Pesquisa Financeira",
        func=search_tool.run,
        description="Use para buscar informações sobre investimentos e mercado financeiro"
    ),
    Tool(
        name="Análise Financeira",
        func=AnaliseFinanceiraTool()._run,
        description="Use para analisar investimentos e dar conselhos"
    ),
    Tool(
        name="Cálculo Financeiro",
        func=JurosCompostosTool()._run,
        description="Use para calcular juros compostos e projeções"
    )
]

# Template para o agente financeiro
prompt_financeiro = ChatPromptTemplate.from_messages([
    ("system", "Você é um consultor financeiro experiente e inteligente. Use as ferramentas disponíveis para dar conselhos precisos sobre investimentos, mercado financeiro e cálculos financeiros."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# NOVO: Criar agente com sintaxe moderna
agent_financeiro = create_openai_functions_agent(llm, tools_financeiras, prompt_financeiro)

# NOVO: Executor do agente
agent_executor_financeiro = AgentExecutor(
    agent=agent_financeiro,
    tools=tools_financeiras,
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
    verbose=True
)

print("💼 Agente financeiro criado!")
print("🎯 Consultor financeiro inteligente pronto")

In [ ]:
# Testando o agente financeiro
# Vamos ver o consultor em ação!

print("💼 TESTE: AGENTE FINANCEIRO")
print("=" * 50)

consultas_financeiras = [
    "O que é CDB e é um bom investimento?",
    "Calcule quanto eu teria se investisse R$ 1000 por 12 meses",
    "Pesquise sobre as melhores ações para investir em 2024"
]

for i, consulta in enumerate(consultas_financeiras, 1):
    print(f"\n💰 Consulta {i}: {consulta}")

    try:
        # NOVO: Usar agent_executor_financeiro.invoke() em vez de agent_financeiro.run()
        resposta = agent_executor_financeiro.invoke({"input": consulta})

        # NOVO: Acessar o conteúdo da resposta
        print(f"💼 Consultor: {resposta['output'][:300]}...")

    except Exception as e:
        print(f"❌ Erro: {e}")

    print("-" * 40)

### **Na Prática, Meu Consagrado!** 💪

**O que aprendemos sobre Agents:**

1. ✅ **Agents básicos** - IA que pode usar ferramentas
2. ✅ **Ferramentas customizadas** - Swiss Army Knife da IA
3. ✅ **Tomada de decisão** - Agents que escolhem qual ferramenta usar
4. ✅ **Especialização** - Agents para áreas específicas

### **Comparação: Com vs Sem LangChain**

**Sem LangChain**: IA só conversa, não pode fazer ações
**Com LangChain**: IA pode pesquisar, calcular, analisar, tomar decisões

<img src = 'https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Fagent-framework.ad7f5098.png&w=1920&q=75' img>

**🎯 Próximo módulo**: Vamos aprender sobre **Document Loaders** - como fazer a IA ler documentos!

**💡 Resumo do Módulo 5**:
- ✅ Agents básicos e avançados
- ✅ Ferramentas customizadas
- ✅ Tomada de decisão inteligente
- ✅ Especialização por área

**🤖 Agora você tem funcionários inteligentes que podem usar ferramentas!**